# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1VERPjEZcC1XSs4-02aM-DbkNr_yaJVbFjLJxaYQswqA/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_**All code below should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach.**_

## Requirements

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project._

* Code clarity: make sure the code conforms to:
    * [ ] [PEP 8](https://peps.python.org/pep-0008/) - You might find [this resource](https://realpython.com/python-pep8/) helpful as well as [this](https://github.com/dnanhkhoa/nb_black) or [this](https://jupyterlab-code-formatter.readthedocs.io/en/latest/) tool
    * [ ] [PEP 257](https://peps.python.org/pep-0257/)
    * [ ] Break each task down into logical functions
* The following files are submitted for the project (see the project's GDoc for more details):
    * [ ] `README.md`
    * [ ] `requirements.txt`
    * [ ] `.gitignore`
    * [ ] `schema.sql`
    * [ ] 6 query files (using the `.sql` extension), appropriately named for the purpose of the query
    * [x] Jupyter Notebook containing the project (this file!)
* [x] You can edit this cell and add a `x` inside the `[ ]` like this task to denote a completed task

## Project Setup

In [1]:
# all import statements needed for the project, for example:

import requests
import bs4
import re
import pyarrow.parquet as pq
import pandas as pd
import geopandas as gpd
from skimpy import clean_columns
import sqlalchemy as db
import matplotlib.pyplot as plt
import math
import os
import io

In [2]:
# any general notebook setup, like log formatting

In [3]:
# any constants you might need, for example:

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
# add other constants to refer to any local data, e.g. uber & weather
UBER_DATA = "uber_rides_sample.csv"

NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))
ny_lat0, ny_lon0 = NEW_YORK_BOX_COORDS[0]
ny_lat1, ny_lon1 = NEW_YORK_BOX_COORDS[1]

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

## Part 1: Data Preprocessing

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Define a function that calculates the distance between two coordinates in kilometers that **only uses the `math` module** from the standard library.
* [ ] Taxi data:
    * [ ] Use the `re` module, and the packages `requests`, BeautifulSoup (`bs4`), and (optionally) `pandas` to programmatically download the required CSV files & load into memory.
    * You may need to do this one file at a time - download, clean, sample. You can cache the sampling by saving it as a CSV file (and thereby freeing up memory on your computer) before moving onto the next file. 
* [ ] Weather & Uber data:
    * [ ] Download the data manually in the link provided in the project doc.
* [ ] All data:
    * [ ] Load the data using `pandas`
    * [ ] Clean the data, including:
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * (Taxi & Uber data) Remove trips that start and/or end outside the designated [coordinate box](http://bboxfinder.com/#40.560445,-74.242330,40.908524,-73.717047)
    * [ ] (Taxi data) Sample the data so that you have roughly the same amount of data points over the given date range for both Taxi data and Uber data.
* [ ] Weather data:
    * [ ] Split into two `pandas` DataFrames: one for required hourly data, and one for the required daily daya.
    * [ ] You may find that the weather data you need later on does not exist at the frequency needed (daily vs hourly). You may calculate/generate samples from one to populate the other. Just document what you’re doing so we can follow along. 

### Data lookup from taxi_zones.shp

In [4]:
taxi_zone = gpd.read_file(r"taxi_zones.shp")

In [5]:
# Get the center of each polygon
taxi_zone = taxi_zone.to_crs(4326)
taxi_zone["lon"] = taxi_zone.centroid.x
taxi_zone["lat"] = taxi_zone.centroid.y

# make sure that the coodinate is located inside NY bounding box
taxi_zone = taxi_zone[(taxi_zone["lon"] > ny_lon0) & (taxi_zone["lon"] < ny_lon1)]
taxi_zone = taxi_zone[(taxi_zone["lat"] > ny_lon1) & (taxi_zone["lat"] < ny_lat1)]

/tmp/ipykernel_1471/3630155683.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  taxi_zone["lon"] = taxi_zone.centroid.x
/tmp/ipykernel_1471/3630155683.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  taxi_zone["lat"] = taxi_zone.centroid.y


In [6]:
#create dataframe to handle location data
df_taxi_zone = taxi_zone[["LocationID"]].copy()
df_taxi_zone["coordinate"] = pd.Series(zip(taxi_zone.lat, taxi_zone.lon))

df_taxi_zone = df_taxi_zone.dropna()

df_taxi_zone

,LocationID,coordinate
0,1,"(40.69183120640149, -74.17400027276298)"
1,2,"(40.61674529165988, -73.83129854302214)"
2,3,"(40.86447368477543, -73.84742223236718)"
3,4,"(40.72375214158478, -73.97696825691767)"
4,5,"(40.552659286945655, -74.18848410184931)"
...,...,...
257,258,"(40.897932025294715, -73.8522154537012)"
258,259,"(40.74423471780149, -73.90630644197886)"
259,260,"(40.70913894067186, -74.01302277174901)"
260,261,"(40.77593240314995, -73.94651035601467)"


In [101]:
#Validate of NY bounding box (df_location)
"""
lat_result = []
lon_result = []

for data in range(len(df_location.coordinate)):
    lat, lon = df_location["coordinate"].iloc[data]
    
    lat_result.append(lat)
    lon_result.append(lon)

min_lat = min(lat_result)
max_lat = max(lat_result)
min_lon = min(lon_result)
max_lon = max(lon_result)
"""
# Note: NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))
# min_lon, max_lon = (-74.23353354937652, -73.72665537528711)
# min_lat, max_lat = (40.525494581763816, 40.89952906437648)

'\nlat_result = []\nlon_result = []\n\nfor data in range(len(df_location.coordinate)):\n    lat, lon = df_location["coordinate"].iloc[data]\n    \n    lat_result.append(lat)\n    lon_result.append(lon)\n\nmin_lat = min(lat_result)\nmax_lat = max(lat_result)\nmin_lon = min(lon_result)\nmax_lon = max(lon_result)|\n'

### Calculating distance
_**TODO:** Write some prose that tells the reader what you're about to do here._

In [8]:
def calculate_distance(from_coord, to_coord):
    
    result = []
    
    for data in range(max(len(from_coord), len(to_coord))):
        lat1, lon1 = from_coord[data]
        lat2, lon2 = to_coord[data]
                      
        #method to convert a degree value into radians
        lon1 = math.radians(lon1)
        lon2 = math.radians(lon2)
        lat1 = math.radians(lat1)
        lat2 = math.radians(lat2)
        
        #Haversine Formula
        diff_lon = lon2 - lon1 
        diff_lat = lat2 - lat1
        
        a = math.sin(diff_lat // 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(diff_lon // 2)**2
        c = 2 * math.asinh(math.sqrt(a))
        
        # Radius of earth in kilometers. Use 3956 for miles
        r = 6371
                      
        #calculate the result
        distance = r*c
        result.append(distance)
    
    distance_series = pd.Series(result)
                      
    return distance_series

In [73]:
def add_distance_column(dataframe):
    #merge main dataframe with the df_location based on Pick-up Location
    dataframe = dataframe.merge(df_taxi_zone, left_on='PULocationID', right_on='LocationID')
    dataframe = dataframe.rename(columns={'coordinate':'PUCoordinate'})
    
    #merge main dataframe with the df_location based on Drop-off Location
    dataframe = dataframe.merge(df_taxi_zone, left_on='DOLocationID', right_on='LocationID')
    dataframe = dataframe.rename(columns={'coordinate':'DOCoordinate'})

    dataframe = dataframe.drop(columns=["LocationID_x","LocationID_y"])
    
    dataframe["distance"] = calculate_distance(dataframe["PUCoordinate"], dataframe["DOCoordinate"])
    
    return dataframe["distance"]

### Processing Taxi Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [10]:
def find_taxi_csv_urls():
    #Get taxi HTML
    response = requests.get(TAXI_URL)
    html = response.content
    
    #Find taxi parquet links
    soup = bs4.BeautifulSoup(html, 'html.parser')
    result = [a['href'] for a in soup.find_all('a', title="Yellow Taxi Trip Records")]
    return result

In [11]:
def get_and_clean_month_taxi_data(url):
    dataframe = pd.DataFrame(None)
    match = re.search(r"\w+2022-01.parquet", url) #keep in mind that currently we only take 2022-01 data to make the preprocessing more efficient, we can update the REGEX once we wrapped up all the sections
    if match is None:
        pass
    else:
        response = requests.get(url)
        
        with open(match.group(), "wb") as f:
            for chunk in response.iter_content(chunk_size=1024): 
                if chunk:
                    f.write(chunk)
                    
        with open(match.group(), "r") as f:
            dataframe = pd.read_parquet(match.group(), engine='pyarrow')
            
    return dataframe

In [74]:
def get_and_clean_taxi_data():
    all_taxi_dataframes = []
    
    all_csv_urls = find_taxi_csv_urls()
    for csv_url in all_csv_urls:
        # maybe: first try to see if you've downloaded this exact
        # file already and saved it before trying again
        dataframe = get_and_clean_month_taxi_data(csv_url)
        
        all_taxi_dataframes.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    taxi_data = pd.concat(all_taxi_dataframes)
    
    taxi_data = taxi_data.dropna()
    taxi_data["store_and_fwd_flag"].replace(["Y", "N"],
                                             [1,0], inplace= True)
    
    return taxi_data

### Processing Uber Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [13]:
def load_and_clean_uber_data(csv_file):
    df_uber = pd.read_csv(csv_file)
    
    #renaming old df without creating a copy 
    df_uber.rename(columns={'key': 'date_time', 'Unnamed: 0': 'location_id'}, inplace=True)
    
    #constructing new columns as a function input
    df_uber["pickup_coordinate"] = list(zip(df_uber.pickup_latitude, df_uber.pickup_longitude))
    df_uber["dropoff_coordinate"] = list(zip(df_uber.dropoff_latitude, df_uber.dropoff_longitude))
    
    #calculate distance
    df_uber["distance"] = calculate_distance(df_uber["pickup_coordinate"], df_uber["dropoff_coordinate"])
    
    #make sure that the coordinate is located inside the NY bounding box
    df_uber = df_uber[(df_uber["pickup_longitude"] > ny_lon0) & (df_uber["pickup_longitude"] < ny_lon1)]
    df_uber = df_uber[(df_uber["dropoff_longitude"] > ny_lon0) & (df_uber["dropoff_longitude"] < ny_lon1)]
    df_uber = df_uber[(df_uber["pickup_latitude"] > ny_lat0) & (df_uber["pickup_latitude"] < ny_lat1)]
    df_uber = df_uber[(df_uber["dropoff_latitude"] > ny_lat0) & (df_uber["dropoff_latitude"] < ny_lat1)]
    
    return df_uber

In [14]:
def get_uber_data():
    uber_dataframe = load_and_clean_uber_data(UBER_DATA)
    uber_dataframe = uber_dataframe.drop(columns=["pickup_coordinate","dropoff_coordinate",
                                                  "pickup_longitude", "pickup_latitude", 
                                                  "dropoff_longitude", "dropoff_latitude"])
    
    return uber_dataframe

### Processing Weather Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [15]:
def clean_month_weather_data_hourly(csv_file):
    df_weather = pd.read_csv(csv_file, low_memory = False)
    weather_columns = df_weather.columns.tolist()
    result = ['STATION', 'DATE', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'NAME', 'REPORT_TYPE', 'SOURCE']
    
    for column in weather_columns:
        match = re.search(r"Hourly", column)
        if match is None:
            pass
        else:
            result.append(column)
    
    df_weather_hourly = df_weather[result]
    
    return df_weather_hourly

In [16]:
def clean_month_weather_data_daily(csv_file):
    df_weather = pd.read_csv(csv_file, low_memory = False)
    weather_columns = df_weather.columns.tolist()
    result = ['STATION', 'DATE', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'NAME', 'REPORT_TYPE', 'SOURCE']
    
    for column in weather_columns:
        match = re.search(r"Daily", column)
        if match is None:
            pass
        else:
            result.append(column)
    
    df_weather_daily = df_weather[result]
    
    return df_weather_daily

In [17]:
def load_and_clean_weather_data():
    hourly_dataframes = []
    daily_dataframes = []
    
    # add some way to find all weather CSV files
    weather_csv_files = []
    list_dir = os.listdir() #list all file within a directory and store in an array

    for file in list_dir:
        match = re.search(r"(2009|201[012345])_weather.csv", file)
        if match is None:
            pass
        else:
            weather_csv_files.append(match.group())
    
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    
    return hourly_data, daily_data

### Process All Data

_This is where you can actually execute all the required functions._

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [89]:
taxi_data = get_and_clean_taxi_data()
taxi_data["distance"] = add_distance_column(taxi_data)
uber_data = get_uber_data()
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()

## Part 2: Storing Cleaned Data

_Write some prose that tells the reader what you're about to do here._

In [20]:
engine = db.create_engine(DATABASE_URL)
engine

Engine(sqlite:///project.db)

In [95]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS hourly_weather
(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    STATION INTEGER,
    DATE TEXT,
    LATITUDE REAL,
    LONGITUDE REAL,
    ELEVATION REAL,
    NAME TEXT,
    REPORT_TYPE TEXT,
    SOURCE TEXT,
    HourlyAltimeterSetting REAL,
    HourlyDewPointTemperature REAL,
    HourlyDryBulbTemperature REAL,
    HourlyPrecipitation REAL,
    HourlyPresentWeatherType REAL,
    HourlyPressureChange REAL,
    HourlyPressureTendency REAL,
    HourlyRelativeHumidity REAL,
    HourlySkyConditions REAL,
    HourlySeaLevelPressure REAL,
    HourlyStationPressure REAL,
    HourlyVisibility REAL,
    HourlyWetBulbTemperature REAL,
    HourlyWindDirection REAL,
    HourlyWindGustSpeed REAL,
    HourlyWindSpeed REAL
    )
"""

DAILY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS daily_weather
(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    STATION INTEGER,
    DATE TEXT,
    LATITUDE REAL,
    LONGITUDE REAL,
    ELEVATION REAL,
    NAME TEXT,
    REPORT_TYPE TEXT,
    SOURCE TEXT,
    DailyAverageDewPointTemperature REAL,
    DailyAverageDryBulbTemperature REAL,
    DailyAverageRelativeHumidity REAL,
    DailyAverageSeaLevelPressure REAL, 
    DailyAverageStationPressure REAL,  
    DailyAverageWetBulbTemperature REAL,
    DailyAverageWindSpeed REAL,        
    DailyCoolingDegreeDays REAL,       
    DailyDepartureFromNormalAverageTemperature REAL,
    DailyHeatingDegreeDays REAL,        
    DailyMaximumDryBulbTemperature REAL,
    DailyMinimumDryBulbTemperature REAL,
    DailyPeakWindDirection REAL,        
    DailyPeakWindSpeed REAL,            
    DailyPrecipitation REAL,            
    DailySnowDepth REAL,                
    DailySnowfall REAL,                 
    DailySustainedWindDirection REAL,   
    DailySustainedWindSpeed REAL,       
    DailyWeather TEXT
    )
"""

TAXI_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS taxi_trips
(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    VendorID INTEGER PRIMARY KEY AUTOINCREMENT,
    tpep_pickup_datetime TEXT,
    tpep_dropoff_datetime TEXT,
    passenger_count REAL,
    trip_distance REAL,
    RatecodeID REAL,
    store_and_fwd_flag INTEGER,
    PULocationID INTEGER,
    DOLocationID INTEGER,
    payment_type INTEGER,
    fare_amount REAL,
    extra REAL,
    mta_tax REAL,
    tip_amount REAL,
    tolls_amount REAL,
    improvement_surcharge REAL,
    total_amount REAL,
    congestion surcharge REAL,
    airport_fee REAL,
    distance REAL
)
"""

UBER_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS uber_trips
(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    location_id BIGINT,
    date_time TEXT,
    fare_amount REAL,
    pickup_datetime TEXT,
    passenger_count INTEGER,
    distance REAL
)
"""

In [96]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [97]:
# create the tables with the schema files
with engine.connect() as connection:
    connection.execute(UBER_TRIPS_SCHEMA)
    connection.close()

### Add Data to Database

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [98]:
def write_dataframes_to_table(table_to_df_dict):
    for key, value in table_to_df_dict.items():
        value.to_sql(key, engine, if_exists='append', index=False)

In [99]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_weather_data,
    "daily_weather": daily_weather_data
}

In [100]:
write_dataframes_to_table(map_table_name_to_dataframe)

## Part 3: Understanding the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] For 01-2009 through 06-2015, what hour of the day was the most popular to take a yellow taxi? The result should have 24 bins.
* [ ] For the same time frame, what day of the week was the most popular to take an uber? The result should have 7 bins.
* [ ] What is the 95% percentile of distance traveled for all hired trips during July 2013?
* [ ] What were the top 10 days with the highest number of hired rides for 2009, and what was the average distance for each day?
* [ ] Which 10 days in 2014 were the windiest, and how many hired trips were made on those days?
* [ ] During Hurricane Sandy in NYC (Oct 29-30, 2012) and the week leading up to it, how many trips were taken each hour, and for each hour, how much precipitation did NYC receive and what was the sustained wind speed?

In [ ]:
def write_query_to_file(query, outfile):
    raise NotImplemented()

### Query N

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each query_

In [ ]:
QUERY_N = """
TODO
"""

In [ ]:
engine.execute(QUERY_N).fetchall()

In [ ]:
write_query_to_file(QUERY_N, "some_descriptive_name.sql")

## Part 4: Visualizing the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Create an appropriate visualization for the first query/question in part 3
* [ ] Create a visualization that shows the average distance traveled per month (regardless of year - so group by each month). Include the 90% confidence interval around the mean in the visualization
* [ ] Define three lat/long coordinate boxes around the three major New York airports: LGA, JFK, and EWR (you can use bboxfinder to help). Create a visualization that compares what day of the week was most popular for drop offs for each airport.
* [ ] Create a heatmap of all hired trips over a map of the area. Consider using KeplerGL or another library that helps generate geospatial visualizations.
* [ ] Create a scatter plot that compares tip amount versus distance.
* [ ] Create another scatter plot that compares tip amount versus precipitation amount.

_Be sure these cells are executed so that the visualizations are rendered when the notebook is submitted._

### Visualization N

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each visualization._

_The example below makes use of the `matplotlib` library. There are other libraries, including `pandas` built-in plotting library, kepler for geospatial data representation, `seaborn`, and others._

In [ ]:
# use a more descriptive name for your function
def plot_visual_n(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_n():
    # Query SQL database for the data needed.
    # You can put the data queried into a pandas dataframe, if you wish
    raise NotImplemented()

In [ ]:
some_dataframe = get_data_for_visual_n()
plot_visual_n(some_dataframe)